#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [1]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [20]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [3]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [4]:
image_train.head()

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 10

Data:
+-----+----------------------+------------+-------------------------------+
|  id |        image         |   label    |         deep_features         |
+-----+----------------------+------------+-------------------------------+
|  24 | Height: 32 Width: 32 |    bird    | [0.242871761322, 1.0954537... |
|  33 | Height: 32 Width: 32 |    cat     | [0.525087952614, 0.0, 0.0,... |
|  36 | Height: 32 Width: 32 |    cat     | [0.566015958786, 0.0, 0.0,... |
|  70 | Height: 32 Width: 32 |    dog     | [1.12979578972, 0.0, 0.0, ... |
|  90 | Height: 32 Width: 32 |    bird    | [1.71786928177, 0.0, 0.0, ... |
|  97 | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
| 107 | Height: 32 Width: 32 |    dog     | [0.0, 0.0, 0.220677852631,... |
| 121 | Height: 32 Width: 32 |    bird    | [0.0, 0.23753464222, 0.0, ... |
| 136 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 138 | Height: 32 Width: 32 |    bird    | [0.658935725689, 0.0, 0.0,... |
+-----+----------------------+------------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [73.0, 77.0, 58.0, 71.0, 6... |
| [7.0, 5.0, 8.0, 7.0, 5.0, ... |
| [169.0, 122.0, 65.0, 131.0... |
| [154.0, 179.0, 152.0, 159.... |
| [216.0, 195.0, 180.0, 201.... |
| [33.0, 44.0, 27.0, 29.0, 4... |
| [97.0, 51.0, 31.0, 104.0, ... |
| [93.0, 96.0, 88.0, 102.0, ... |
| [35.0, 59.0, 53.0, 36.0, 5... |
| [205.0, 193.0, 195.0, 200.... |
+-------------------------------+
[10 rows x 5 columns]

#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [5]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [6]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [7]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 22.076ms     |

| Done         |         | 100         | 252.178ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |       384       |      0.0      |  1   |
|      0      |       6910      | 36.9403137951 |  2   |
|      0      |      39777      | 38.4634888975 |  3   |
|      0      |      36870      | 39.7559623119 |  4   |
|      0      |      41734      | 39.7866014148 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

We are going to create a simple function to view the nearest neighbors to save typing:

In [8]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [9]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 11.666ms     |

| Done         |         | 100         | 243.24ms     |

+--------------+---------+-------------+--------------+

In [10]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

##Finding similar images to a car

In [11]:
car = image_train[8:9]
car['image'].show()

In [12]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 8.907ms      |

| Done         |         | 100         | 241.341ms    |

+--------------+---------+-------------+--------------+

#Just for fun, let's create a lambda to find and show nearest neighbor images

In [13]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [14]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 10.852ms     |

| Done         |         | 100         | 237.927ms    |

+--------------+---------+-------------+--------------+

In [15]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 13.172ms     |

| Done         |         | 100         | 233.228ms    |

+--------------+---------+-------------+--------------+

In [16]:
dog_image = image_train[image_train['label'] == 'dog' ]
cat_image = image_train[image_train['label'] == 'cat' ]
automobile_image = image_train[image_train['label'] == 'automobile' ]
bird_image = image_train[image_train['label'] == 'bird' ]

In [18]:
dog_model = graphlab.nearest_neighbors.create(dog_image,features=['deep_features'],
                                             label='id')
cat_model = graphlab.nearest_neighbors.create(cat_image,features=['deep_features'],
                                             label='id')
automobile_model = graphlab.nearest_neighbors.create(automobile_image,features=['deep_features'],
                                             label='id')
bird_model = graphlab.nearest_neighbors.create(bird_image,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [21]:
cat = image_test[0:1]

cat_neighbors = get_images_from_ids(cat_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 39.484ms     |

| Done         |         | 100         | 116.754ms    |

+--------------+---------+-------------+--------------+

In [57]:
cat_neighbors['image'].show()

In [24]:
first_test = image_test[0:1]
dog_neighbors = get_images_from_ids(dog_model.query(first_test))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 24.962ms     |

| Done         |         | 100         | 108.217ms    |

+--------------+---------+-------------+--------------+

In [25]:
dog_neighbors['image'].show()

In [28]:
cat_model.distance

[[['deep_features'], 'euclidean', 1.0]]

In [29]:
cat_neighbors

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 5

Data:
+-------+----------------------+-------+-------------------------------+
|   id  |        image         | label |         deep_features         |
+-------+----------------------+-------+-------------------------------+
|  331  | Height: 32 Width: 32 |  cat  | [0.0, 0.0, 0.510963916779,... |
| 16289 | Height: 32 Width: 32 |  cat  | [0.964287519455, 0.0, 0.0,... |
| 25713 | Height: 32 Width: 32 |  cat  | [0.536971271038, 0.0, 0.0,... |
| 32139 | Height: 32 Width: 32 |  cat  | [1.29409468174, 0.0, 0.0, ... |
| 45646 | Height: 32 Width: 32 |  cat  | [0.983677506447, 0.0, 0.0,... |
+-------+----------------------+-------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [45.0, 65.0, 92.0, 72.0, 9... |
| [215.0, 219.0, 231.0, 215.... |
| [228.0, 222.0, 236.0, 224.... |
| [217.0, 220.0, 205.0, 221.... |
| [51.0, 42.0, 26.0, 56.0, 4... |
+-------------------------------+
[5 rows x 5 columns]

In [30]:
cat_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 27.293ms     |

| Done         |         | 100         | 129.093ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [31]:
dog_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 28.129ms     |

| Done         |         | 100         | 135.78ms     |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16976      | 37.4642628784 |  1   |
|      0      |      13387      | 37.5666832169 |  2   |
|      0      |      35867      | 37.6047267079 |  3   |
|      0      |      44603      | 37.7065585153 |  4   |
|      0      |       6094      | 38.5113254907 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [34]:
#, image_test_dog, image_test_bird, image_test_automobile
image_test_cat = image_test[image_test['label'] == 'cat' ]
image_test_dog = image_test[image_test['label'] == 'dog' ]
image_test_automobile = image_test[image_test['label'] == 'automobile' ]
image_test_bird = image_test[image_test['label'] == 'bird' ]
 

In [35]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 433.351ms    |

| Done         | 509000  | 100         | 460.317ms    |

+--------------+---------+-------------+--------------+

In [37]:
#dog-automobile
#dog-bird
#dog-dog
dog_automobile_neighbors = automobile_model.query(image_test_dog, k=1)
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 128000  | 25.1473     | 447.205ms    |

| Done         | 509000  | 100         | 489.223ms    |

+--------------+---------+-------------+--------------+

In [38]:
#dog-automobile	dog-bird	dog-cat	dog-dog
distances_sframe = graphlab.SFrame({'dog-automobile': dog_automobile_neighbors['distance'],'dog-bird': dog_bird_neighbors['distance'],
                             'dog-cat': dog_cat_neighbors['distance'],'dog-dog': dog_dog_neighbors['distance']})

In [40]:
distances_sframe.show()

In [42]:
distances_sframe

Columns:
	dog-automobile	float
	dog-bird	float
	dog-cat	float
	dog-dog	float

Rows: 1000

Data:
+----------------+---------------+---------------+---------------+
| dog-automobile |    dog-bird   |    dog-cat    |    dog-dog    |
+----------------+---------------+---------------+---------------+
| 41.9579761457  | 41.7538647304 | 36.4196077068 | 33.4773590373 |
| 46.0021331807  | 41.3382958925 | 38.8353268874 | 32.8458495684 |
| 42.9462290692  | 38.6157590853 | 36.9763410854 | 35.0397073189 |
| 41.6866060048  | 37.0892269954 | 34.5750072914 | 33.9010327697 |
| 39.2269664935  |  38.272288694 |  34.778824791 | 37.4849250909 |
| 40.5845117698  | 39.1462089236 | 35.1171578292 |  34.945165344 |
| 45.1067352961  |  40.523040106 | 40.6095830913 | 39.0957278345 |
| 41.3221140974  | 38.1947918393 | 39.9036867306 | 37.7696131032 |
| 41.8244654995  | 40.1567131661 | 38.0674700168 | 35.1089144603 |
| 45.4976929401  | 45.5597962603 | 42.7258732951 | 43.2422832585 |
+----------------+---------------+---------------+---------------+
[1000 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [44]:
distances_sframe['dog-cat']

dtype: float
Rows: 1000
[36.41960770675437, 38.83532688735544, 36.97634108541545, 34.575007291446106, 34.778824791016596, 35.11715782924591, 40.60958309132649, 39.90368673062214, 38.06747001682118, 42.725873295060325, 40.07334854106313, 31.6633518591745, 37.71246671323846, 39.09035764803644, 49.57967416577316, 36.077387742837814, 36.50902452301206, 44.95243961942733, 33.4368280817404, 34.32455551725352, 34.71470962990657, 33.23747741685298, 34.425383319771534, 34.51304231593018, 37.984239884361635, 41.93783581252019, 39.47553315875406, 37.61347349608415, 36.46096099330766, 32.54458323133461, 36.831355406189786, 37.8590258146085, 41.781173252799235, 35.53207404381254, 33.12115384662451, 34.95202883174986, 33.21653861184684, 30.27480523958585, 35.2565724242629, 36.64845825198308, 33.676383475677746, 32.426395075244436, 39.64035075870523, 44.351332458546096, 38.26287205696334, 37.194245743826976, 38.41620975302145, 33.474756973026714, 35.284795792594785, 43.05319736547563, 31.586898628452

In [50]:
def is_dog_correct(row):
    if row["dog-dog"] < row["dog-cat"] and row["dog-dog"] < row["dog-bird"] and row["dog-dog"] < row["dog-automobile"]:
        result = 1
    else:
        result = 0
    return result


In [52]:
distances_sframe.apply(is_dog_correct).sum()

678

In [54]:
678/len(distances_sframe.apply(is_dog_correct))

0

In [55]:
distances_sframe.apply(is_dog_correct)

dtype: int
Rows: 1000
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ... ]

In [56]:
len(distances_sframe.apply(is_dog_correct))

1000